# 进展报告自动生成

基于项目文件（GitHubClient）调用大模型（LLM）自动生成项目进展报告。

### 调用 OpenAI GPT 大模型

相比 GitHub REST API ，OpenAI 提供的大模型相关 API 迭代速度快，且不够稳定。

**GPT-4 很难能够准确的生成 OpenAI Client 相关代码**。

因此，GitHubSentinel 项目中 LLM 相关调用代码由人类编写😁。


## Prompt 优化测试

基于 `GithubClient` 模块获取的 Repo 最新进展，先在 ChatGPT 中尝试获取可用的提示词（Prompt）方案。

-  **完整的ChatGPT 对话记录【GitHubSentinel 提示词优化】**：https://chatgpt.com/share/28524ea6-2bf3-4ebe-b7d9-9c1ba5f005d2
- 以下测试使用的 LangChain 项目文件为: `./daily_progress/langchain-ai_langchain/2024-08-18.md'`


### ChatGPT（GPT-4） 生成报告

**Langchain-AI/Langchain Daily Progress Report - 2024-08-18**

### 新增功能
1. **Langchain 模块添加**
   - 新增了Langchain Box套件及其文档加载器 (`langchain-box: add langchain box package and DocumentLoader` #25506)
   - 加入了新的社区提供者—Agentic RAG 示例 (`Community: Add Union provider - Agentic RAG example` #25509)
   - 引入了更多的异步测试标准 (`standard-tests[patch]: async variations of all tests` #25501)
   - 引入了对多种区块链的支持 (`community: add supported blockchains to Blockchain Document Loader` #25428)
   
2. **文档与API更新**
   - 更新了多个集成参考文档和Langchain版本的文档 (`docs: `integrations` reference update 9` #25511, `docs 0.3 release` #25459)
   - 增加了新的文档索引和数据加载方式的说明 (`[docs]: more indexing of document loaders` #25500)

### 主要改进
1. **测试与标准化**
   - 添加了更多嵌入标准测试 (`more embeddings standard tests` #25513)
   - 引入了JSON模式的标准测试 (`json mode standard test` #25497)
   - 新增了各种文档加载器的文档 (`[Doc] Add docs for `ZhipuAIEmbeddings`` #25467)

2. **框架和规则改进**
   - 对Langchain核心模块进行了Pydantic解析器修复 (`langchain-core: added pydantic parser fix for issue #24995` #25516)
   - 增加了B(bugbear) ruff规则以提高代码质量 (`core: Add B(bugbear) ruff rules` #25520)

3. **集成与兼容性**
   - 测试了Pydantic 2和Langchain 0.3的兼容性 (`openai[major] -- test with pydantic 2 and langchain 0.3` #25503)
   - 准备了向Pydantic 2迁移的根验证器升级 (`openai[patch]: Upgrade @root_validators in preparation for pydantic 2 migration` #25491)

### 修复问题
1. **错误修复**
   - 修正了文档中的错别字和错误消息 (`docs: Fix typo in openai llm integration notebook` #25492, `docs: fix Agent deprecation msg` #25464)
   - 解决了不同的搜索模式（向量与文本）产生不同结果的问题 (`Chroma search with vector and search with text get different result using the same embedding function` #25517)
   - 修复了使用AzureSearch vectorstore时的文档ID作为键的问题 (`community : [bugfix] Use document ids as keys in AzureSearch vectorstore` #25486)

2. **系统错误与异常处理**
   - 解决了在调用特定链时缺少输入键的错误 (`Raises ValueError: Missing some input keys: {'query'} everytime I invoke 'GraphCypherQAChain.from_llm' chain with query present as input keys` #25476)
   - 修正了未知类型 'ToolMessage' 的类型错误 (`TypeError: Got unknown type 'ToolMessage'.` #25490)

---

此简报详细总结了Langchain AI项目在2024年8月18日的最新进展，包括新增功能、主要改进和问题修复，确保团队成员了解最新的项目状态和即将到来的更新。


## 前置依赖 logger 模块

In [1]:
# src/logger.py
from loguru import logger
import sys

# Configure Loguru
logger.remove()  # Remove the default logger
logger.add(sys.stdout, level="DEBUG", format="{time} {level} {message}", colorize=True)
logger.add("logs/app.log", rotation="1 MB", level="DEBUG")

# Alias the logger for easier import
LOG = logger

# Make the logger available for import with the alias
__all__ = ["LOG"]

## LLM Class 

In [2]:
import os
from openai import OpenAI  # 导入OpenAI库用于访问GPT模型
# from logger import LOG  # 导入日志模块（演示时直接导入）

class LLM:
    def __init__(self, model="gpt-3.5-turbo"):
        # 创建一个OpenAI客户端实例
        self.client = OpenAI(
                base_url="https://api.gptsapi.net/v1",
                api_key=os.getenv("OPENAI_KEY")
            )
        # 确定使用的模型版本
        self.model = model
        # 配置日志文件，当文件大小达到1MB时自动轮转，日志级别为DEBUG
        LOG.add("daily_progress/llm_logs.log", rotation="1 MB", level="DEBUG")

    def generate_daily_report(self, markdown_content, dry_run=False):
        # 构建一个用于生成报告的提示文本，要求生成的报告包含新增功能、主要改进和问题修复
        # prompt = f"以下是项目的最新进展，根据功能合并同类项，形成一份简报，至少包含：1）新增功能；2）主要改进；3）修复问题；:\n\n{markdown_content}"
        # system_prompt = f"你是一个定期跟进GitHub开源项目进展的智能助手，负责解读最新进展信息，并归纳总结成中文简报。\n\n你将收到由issues和pull requests组成的最新进展信息，请分析解读后生成一份进展简报，涵盖以下内容：\n\n1.概括总结项目当前重点进展情况。仅针对提供的信息总结，避免套话。\n\n2.根据新增功能、修复问题、优化改进整理和合并相似的issue和pull request。要求整理结果逐条展示，并在每条最后保留issue或PR的number号。"
        system_prompt = f"你是一个善于总结GitHub开源项目进展的中文智能助手。\n\n你将收到由issues和pull requests组成的详细进展信息，请你分析和解读后，归纳总结成一份中文简报，涵盖以下3类内容：1.新增功能；2.修复问题；3.优化改进。\n\n简报内容严格保持markdown格式，务必使用中文总结。每个类别的进展逐条展示，每条进展最后保留issue或PR的number号。\n\n要求整理和合并相似的issue和pull request，相同的number号仅需要在一个类别里。\n\n"
        user_prompt = f"{markdown_content}"
        if dry_run:
            # 如果启用了dry_run模式，将不会调用模型，而是将提示信息保存到文件中
            LOG.info("Dry run mode enabled. Saving prompt to file.")
            with open("daily_progress/prompt.txt", "w+") as f:
                f.write(system_prompt)
                f.write("\n\n")
                f.write(user_prompt)
            LOG.debug("Prompt saved to daily_progress/prompt.txt")
            return "DRY RUN"

        # 日志记录开始生成报告
        LOG.info(f"Starting report generation using model: {self.model}.")
        
        try:
            # 调用OpenAI GPT模型生成报告
            response = self.client.chat.completions.create(
                model=self.model,  # 指定使用的模型版本
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}  # 提交用户角色的消息
                ]
            )
            LOG.debug(f"{self.model} response: {response}")
            # 返回模型生成的内容
            return response.choices[0].message.content
        except Exception as e:
            # 如果在请求过程中出现异常，记录错误并抛出
            LOG.error("An error occurred while generating the report: {}", e)
            raise


## ReportGenerator Class

In [3]:
# src/report_generator.py

import os
from datetime import date, timedelta
# from logger import LOG  # 导入日志模块，用于记录日志信息

class ReportGenerator:
    def __init__(self, llm):
        self.llm = llm  # 初始化时接受一个LLM实例，用于后续生成报告

    def export_daily_progress(self, repo, updates):
        # 构建仓库的日志文件目录
        repo_dir = os.path.join('daily_progress', repo.replace("/", "_"))
        os.makedirs(repo_dir, exist_ok=True)  # 如果目录不存在则创建
        
        # 创建并写入日常进展的Markdown文件
        file_path = os.path.join(repo_dir, f'{date.today()}.md')
        with open(file_path, 'w') as file:
            file.write(f"# Daily Progress for {repo} ({date.today()})\n\n")
            file.write("\n## Issues\n")
            for issue in updates['issues']:
                file.write(f"- {issue['title']} #{issue['number']}\n")
            file.write("\n## Pull Requests\n")
            for pr in updates['pull_requests']:
                file.write(f"- {pr['title']} #{pr['number']}\n")
        return file_path

    def export_progress_by_date_range(self, repo, updates, days):
        # 构建目录并写入特定日期范围的进展Markdown文件
        repo_dir = os.path.join('daily_progress', repo.replace("/", "_"))
        os.makedirs(repo_dir, exist_ok=True)

        today = date.today()
        since = today - timedelta(days=days)  # 计算起始日期
        
        date_str = f"{since}_to_{today}"  # 格式化日期范围字符串
        file_path = os.path.join(repo_dir, f'{date_str}.md')
        
        with open(file_path, 'w') as file:
            file.write(f"# Progress for {repo} ({since} to {today})\n\n")
            file.write("\n## Issues Closed in the Last {days} Days\n")
            for issue in updates['issues']:
                file.write(f"- {issue['title']} #{issue['number']}\n")
            file.write("\n## Pull Requests Merged in the Last {days} Days\n")
            for pr in updates['pull_requests']:
                file.write(f"- {pr['title']} #{pr['number']}\n")
        
        LOG.info(f"Exported time-range progress to {file_path}")  # 记录导出日志
        return file_path

    def generate_daily_report(self, markdown_file_path):
        # 读取Markdown文件并使用LLM生成日报
        with open(markdown_file_path, 'r') as file:
            markdown_content = file.read()

        report = self.llm.generate_daily_report(markdown_content, False)  # 调用LLM生成报告

        report_file_path = os.path.splitext(markdown_file_path)[0] + "_report.md"
        with open(report_file_path, 'w+') as report_file:
            report_file.write(report)  # 写入生成的报告

        LOG.info(f"Generated report saved to {report_file_path}")  # 记录生成报告日志

    def generate_report_by_date_range(self, markdown_file_path, days):
        # 生成特定日期范围的报告，流程与日报生成类似
        with open(markdown_file_path, 'r') as file:
            markdown_content = file.read()

        report = self.llm.generate_daily_report(markdown_content)

        report_file_path = os.path.splitext(markdown_file_path)[0] + f"_report.md"
        with open(report_file_path, 'w+') as report_file:
            report_file.write(report)

        LOG.info(f"Generated report saved to {report_file_path}")  # 记录生成报告日志


## 调用 GPT-3.5-Turbo API 生成报告

In [4]:
# 实例化 LLM，并使用默认的 GPT-3.5-Turbo 模型
llm = LLM()

In [5]:
# 实例化 ReportGenerator
report_generator = ReportGenerator(llm)

In [6]:
# 生成 LangChain 项目最近一日报告
report_generator.generate_daily_report(
    markdown_file_path="daily_progress/langchain-ai_langchain_2024-08-28.md")

2024-08-28T02:29:48.264230-0700 INFO Starting report generation using model: gpt-3.5-turbo.
2024-08-28T02:29:56.281217-0700 DEBUG gpt-3.5-turbo response: ChatCompletion(id='chatcmpl-A19HpikkPwNarGhNnOYtbc2uY9T61', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='# langchain-ai/langchain Daily Progress - 2024-08-28\n\n## 新增功能\n- 引入Amazon OpenSearch Serverless (aoss)作为语义缓存储存选项 #25797\n- 新增Constrained COT评估器 #25796\n- 新增SQL记录管理器 #25795\n- 新增UnstructuredFileLoader支持pdf格式 #25774\n- 增加TablestoreVectorStore #25767\n- 新增Apache Opendal S3文档加载器 #25734\n- 支持在Elastic Search容器中存储的DenseX嵌入 #25744\n- 新增 gaode weather query tool #25654\n\n## 修复问题\n- 修复向量存储器 QdrantVectorStore 类型的自查询检索问题 #25798\n- 修复 bug #25793\n- 修复zep_could存储空间的内存错误 #25791\n- 修复 Moderation Chain 中的 B/C问题 #25778\n- 修复chat模型启动参数默认值和一些使用model provider的问题 #25731\n- 修复deepinfra的模型初始化bug #25727\n\n## 优化改进\n- 修改community：避免“langchain_prompty”中的双重模板 #25777\n- 使PineconeHybridSearchRetriever支持

## 调用 GPT-4-Turbo API 生成报告

In [7]:
# 实例化 LLM，并使用指定的 GPT-4-Turbo 模型
gpt_4 = LLM(model="gpt-4-turbo")

In [8]:
# 实例化 ReportGenerator, 并使用指定的 GPT-4-Turbo 模型
rg_gpt_4 = ReportGenerator(gpt_4)

In [9]:
# 生成 LangChain 项目最近一日报告
rg_gpt_4.generate_daily_report(
    markdown_file_path="daily_progress/langchain-ai_langchain_2024-08-28.md")

2024-08-28T02:31:09.577023-0700 INFO Starting report generation using model: gpt-4-turbo.
2024-08-28T02:31:44.002469-0700 DEBUG gpt-4-turbo response: ChatCompletion(id='chatcmpl-A19J8JTPKFZA3e1e8lhjYRWZ7W277', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="### 新增功能\n- 新增支持自定义查询检索器 `VectorStore` 类型为 `langchain_qdrant.qdrant.QdrantVectorStore` 的功能。 #25798\n- 新增使用 Amazon OpenSearch Serverless (AOSS) 作为语义缓存存储功能。参与者：Issues & PR #25797\n- 新增“受限COT评估器”评估功能。参与者：Issues & PR #25796\n- 新增 SQL 记录管理器。参与者：Issues #25795\n- 新增 PDF 文件支持的无结构文件加载器。 #25774\n- 新增对 `OpenLLM` 增加本地服务器连接的支持，解决了无属性 'client' 的错误。 #25772\n- 新增 `TablestoreVectorStore` 为存储库增加表存储的支持。参与者：Issues & PR #25767\n- 新增对 `Apache Opendal S3 document loader` 文档加载器的支持，增强云存储文件处理功能。参与者：Issues & PR #25734\n- 新增多元素富文本的正确处理支持。参与者：Issues & PR #25762\n- 新增对 DenseX Embeddings 在 Elasticsearch 容器中的存储支持。 #25744\n- 新增对 Neo4j 向量嵌入的维度检查可选支持。参与者：Issues & PR #25737\n- 新增异步支持Redis聊天信息历史。参与者：Issues & PR #2573

### Homework: 与ChatGPT深度对话，尝试使用 System role 提升报告质量和稳定性